In [3]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Dense
from keras.models import Model, Sequential


from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork


#  Import network from file

In [2]:
import pickle

# WRITE
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'wb') as file:
#     pickle.dump(network, file)

# READ
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'rb') as file:
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_biogrid.pickle', 'rb') as file:
    network = pickle.load(file)
    network.remove_extra_nodes()
#     network.node_list = network.all_nodes
#     node_list = network.node_list

In [3]:
# READ edgelists
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_test_edges.pickle', 'rb') as file:
    test_edges_dict = pickle.load(file)
    
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_val_edges.pickle', 'rb') as file:
    val_edges_dict = pickle.load(file)

# Load training data

In [3]:
X, y = network.multi_omics_data.load_data(modalities=["MIR", "GE"])

In [4]:
X["MIR"].shape

(460, 1870)

In [5]:
transcripts = network.multi_omics_data.LNC.get_genes_info()["Transcript sequence"]

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:291: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.HGNC_lncrna_info.groupby("symbol").first(), on="symbol",
/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6336: FutureWarning: 'Gene Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  rsuffix=rsuffix, sort=sort)


In [6]:
transcripts[transcripts.notna()].map(len).describe()

count    11769.000000
mean       882.826493
std        988.346422
min         60.000000
25%        447.000000
50%        568.000000
75%        833.000000
max      37852.000000
Name: Transcript sequence, dtype: float64

# Training Source Target Graph Embedding

In [4]:
from keras.layers import Input, Conv1D, Lambda, Dot, Dense, Flatten, MaxPooling1D, Lambda
from keras.layers import LSTM, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K

from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy

from keras.utils import to_categorical


def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)


In [5]:
K.clear_session()
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

In [6]:
# node_features_size = X["MIR"].shape[0]
input_shape = (None, 4)
_d = 128
# n_nodes = len(network.node_list)

In [32]:
# with tf.name_scope('inputs'):
E_ij = Input(shape=(), name="E_ij")
#     input_i = tf.placeholder(tf.float32, shape=input_shape, name="input_i")
#     input_j = tf.placeholder(tf.float32, shape=input_shape, name="input_j")
input_seq_i = Input(batch_shape=(1, None, 4), name="input_i")
input_seq_j = Input(batch_shape=(1, None, 4), name="input_j")
# is_directed = tf.placeholder(tf.bool, name="is_directed")
is_directed = tf.placeholder(dtype=tf.bool, name="is_directed")
i = tf.Variable(int, name="i", trainable=False)
j = tf.Variable(int, name="j", trainable=False)

In [33]:
#build convnet to use in each siamese 'leg'
lstm = Sequential()
lstm.add(LSTM(128, input_shape=(None, 4), return_sequences=False))
lstm.add(Dense(_d, activation='sigmoid'))

# encode each of the two inputs into a vector with the convnet
encoded_i = lstm(input_seq_i)
encoded_j = lstm(input_seq_j)
print("encoded_i", encoded_i, "\nencoded_j", encoded_j)


#layer to merge two encoded inputs with the l1 distance between them
# L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
# #call this layer on list of two input tensors.
# L1_distance = L1_layer([encoded_i, encoded_j])
# prediction = Dense(1, activation='sigmoid',bias_initializer=b_init)(L1_distance)


# # output = Lambda(lambda tup: 
# #                 is_directed*tup[0] + (1-is_directed)*tup[1])([output_directed, output_undirected])
# conditional_layer = Lambda(lambda x: K.switch(x[0], K.sigmoid(x[1]), K.sigmoid(x[2])))
# conditional_output = conditional_layer([is_directed, dot_directed, dot_undirected])
# print("conditional_output", conditional_output)

siamese_net = Model(inputs=[input_seq_i, input_seq_j, is_directed], outputs=[encoded_i, encoded_j])

encoded_i Tensor("sequential_12/dense_13/Sigmoid:0", shape=(1, 128), dtype=float32) 
encoded_j Tensor("sequential_12_1/dense_13/Sigmoid:0", shape=(1, 128), dtype=float32)


ValueError: Input tensors to a Model must come from `keras.layers.Input`. Received: Tensor("is_directed_2:0", dtype=bool) (missing previous layer metadata).

In [34]:
def contrastive_loss(encoded_i, encoded_j, E_ij, is_directed):
    dot_directed = Dot(axes=1)([encoded_i[:, 0:int(_d/2)], encoded_j[:, int(_d/2):_d]])
    dot_undirected = Dot(axes=1)([encoded_i, encoded_j])

    distance = -K.log(K.switch(is_directed, K.sigmoid(dot_directed), K.sigmoid(dot_undirected)))

    loss = (1-E_ij)*1/2*K.square(distance) + E_ij*1/2*K.square(K.maximum(0.0, 0.5 - distance))
    return loss

In [35]:
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss=contrastive_loss(encoded_i, encoded_j, E_ij, is_directed), optimizer=Adam(0.00006))

siamese_net.count_params()


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

# Data Generator

In [33]:
from moge.network.data_generator import DataGenerator

In [ ]:
generator = DataGenerator(network.node_list, labels=network.node_list, network=network, 
                          batch_size=1, dim=(None, 4), n_channels=4,n_classes=10, shuffle=True)

In [ ]:
siamese_net.fit_generator()

In [32]:
X, y = next(train_generator())
print(X.shape, y.shape)

(1, 88, 4) (1, 88, 1)


In [31]:
def train_generator():
    while True:
        sequence_length = np.random.randint(10, 100)
        x_train = np.random.random((1, sequence_length, 4))
        # y_train will depend on past 5 timesteps of x
        y_train = x_train[:, :, 0]
        for i in range(1, 2):
            y_train[:, i:] += x_train[:, :-i, i]
        y_train = to_categorical(y_train > 2.5)
        yield x_train, y_train

In [126]:
with tf.name_scope('siamese'):
    siamese = Dense(128, activation='relu')(N_i)
    siamese = Dense(_d, activation='relu')(siamese)
    
    emb_c_i = siamese(N_i)
    emb_c_i = siamese(N_j)


In [127]:
with tf.name_scope('embedding'):
    emb_s = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_t", trainable=True)

    emb_t = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_s", trainable=True)

    emb_c = tf.concat([emb_s, emb_t], axis=1, name="emb_concat")

In [128]:
emb_s = emb_s[i].assign(tf.reshape(siamese[:, 0 : int(_d/2)], [-1]))
emb_t = emb_t[i].assign(tf.reshape(siamese[:, int(_d/2) : _d], [-1]))

# with tf.control_dependencies([emb_s[i].assign(emb_c_i)]):
#     emb_s = tf.identity(emb_s)

In [ ]:
sess.as_default()
K.set_session(sess)
session.run(init_op)